# TASK №1


In [256]:
from ultralytics import YOLO
import cv2
import numpy as np
import os

In [257]:
# Загрузка модели YOLOv8
model = YOLO('yolov8n.pt')

# Список цветов для различных классов
colors = [
    (255, 0, 0), (0, 255, 0), (0, 0, 255), (255, 255, 0), (0, 255, 255),
    (255, 0, 255), (192, 192, 192), (128, 128, 128), (128, 0, 0), (128, 128, 0),
    (0, 128, 0), (128, 0, 128), (0, 128, 128), (0, 0, 128), (72, 61, 139),
    (47, 79, 79), (47, 79, 47), (0, 206, 209), (148, 0, 211), (255, 20, 147)
]


In [258]:
# Функция для обработки изображения
def process_image(image_path):
    # Загрузка изображения
    image = cv2.imread(image_path)
    results = model(image)[0]
    
    # Получение оригинального изображения и результатов
    image = results.orig_img
    classes_names = results.names
    classes = results.boxes.cls.cpu().numpy()
    boxes = results.boxes.xyxy.cpu().numpy().astype(np.int32)

    # Подготовка словаря для группировки результатов по классам
    grouped_objects = {}

    # Рисование рамок и группировка результатов
    for class_id, box in zip(classes, boxes):
        if class_id==0:    
            class_name = classes_names[int(class_id)]
            color = colors[int(class_id) % len(colors)]  # Выбор цвета для класса
            if class_name not in grouped_objects:
                grouped_objects[class_name] = []
            grouped_objects[class_name].append(box)

            # Рисование рамок на изображении
            x1, y1, x2, y2 = box
            cv2.rectangle(image, (x1, y1), (x2, y2), color, 2)
            cv2.putText(image, class_name, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

    # Сохранение измененного изображения
    new_image_path = os.path.splitext(image_path)[0] + '_yolo' + os.path.splitext(image_path)[1]
    cv2.imwrite(new_image_path, image)

    # Сохранение данных в текстовый файл
    text_file_path = os.path.splitext(image_path)[0] + '_data.txt'
    with open(text_file_path, 'w') as f:
        for class_name, details in grouped_objects.items():
            f.write(f"{class_name}:\n")
            for detail in details:
                f.write(f"Coordinates: ({detail[0]}, {detail[1]}, {detail[2]}, {detail[3]})\n")

    count_of_persons = len([el for el in classes if el == 0])
    
    print(f"Count of persons: {count_of_persons}")
    print(f"Processed {image_path}:")
    print(f"Saved bounding-box image to {new_image_path}")
    print(f"Saved data to {text_file_path}")
    return results

In [259]:

res = process_image('data/720x.png')


0: 384x640 7 persons, 1 motorcycle, 36.0ms
Speed: 7.0ms preprocess, 36.0ms inference, 5.0ms postprocess per image at shape (1, 3, 384, 640)
Count of persons: 7
Processed data/720x.png:
Saved bounding-box image to data/720x_yolo.png
Saved data to data/720x_data.txt


#  TASK №2

In [260]:
# Функция для обработки изображения
def process_image(image_path):
    # Загрузка изображения
    image = cv2.imread(image_path)
    results = model(image)[0]
    
    # Получение оригинального изображения и результатов
    image = results.orig_img
    classes_names = results.names
    classes = results.boxes.cls.cpu().numpy()
    boxes = results.boxes.xyxy.cpu().numpy().astype(np.int32)

    # Подготовка словаря для группировки результатов по классам
    grouped_objects = {}

    # Рисование рамок и группировка результатов
    for class_id, box in zip(classes, boxes):
        if class_id==0:    
            class_name = classes_names[int(class_id)]
            color = colors[int(class_id) % len(colors)]  # Выбор цвета для класса
            if class_name not in grouped_objects:
                grouped_objects[class_name] = []
                
            x1, y1, x2, y2 = box
            center_x = (x1 + x2) // 2
            center_y = (y1 + y2) // 2
            grouped_objects[class_name].append((box, (center_x, center_y)))
            
            cv2.rectangle(image, (x1, y1), (x2, y2), color, 2)
            cv2.putText(image, class_name, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

    # Сохранение измененного изображения
    new_image_path = os.path.splitext(image_path)[0] + '_yolo' + os.path.splitext(image_path)[1]
    cv2.imwrite(new_image_path, image)

    # Сохранение данных в текстовый файл
    text_file_path = os.path.splitext(image_path)[0] + '_data.txt'
    with open(text_file_path, 'w') as f:
        for class_name, details in grouped_objects.items():
            f.write(f"{class_name}:\n")
            for box, center in details:
                f.write(f"Coordinates: ({box[0]}, {box[1]}, {box[2]}, {box[3]}) Center: ({center[0]}, {center[1]})\n")

    count_of_persons = len([el for el in classes if el == 0])
    
    print(f"Count of persons: {count_of_persons}")
    print(f"Processed {image_path}:")
    print(f"Saved bounding-box image to {new_image_path}")
    print(f"Saved data to {text_file_path}")
    return results, grouped_objects

In [261]:

res, objects = process_image('data/720x.png')


0: 384x640 7 persons, 1 motorcycle, 30.0ms
Speed: 4.0ms preprocess, 30.0ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)
Count of persons: 7
Processed data/720x.png:
Saved bounding-box image to data/720x_yolo.png
Saved data to data/720x_data.txt


In [263]:
centers = np.array([objects['person'][el][1]  for el in range(len(objects['person']))])

groups_of_persons= {}

num_rectangles = centers.shape[0]
distance_matrix = np.zeros((num_rectangles, num_rectangles))
counter=-1
for i in range(num_rectangles):
    for j in range(num_rectangles):
        if i != j:
            distance_matrix[i, j] = np.linalg.norm(centers[i] - centers[j])
            if distance_matrix[i,j]<100:
                all_values = [item for sublist in groups_of_persons.values() for item in sublist]
                if i not in all_values  and j not in all_values:
                    counter+=1
                    groups_of_persons.update({f"Group №{counter}" :[i,  j]})
                else:
                    groups_of_persons[f"Group №{counter}"].extend([i, j])
            

# Шаг 3: Вывод матрицы расстояний
print("Матрица расстояний:")
print(distance_matrix)

Матрица расстояний:
[[          0      514.57      91.302      70.612      536.62      83.024      117.78]
 [     514.57           0      424.04      457.96      151.16      445.49      396.83]
 [     91.302      424.04           0      60.017      454.09      57.035       28.16]
 [     70.612      457.96      60.017           0      469.95      13.153      73.246]
 [     536.62      151.16      454.09      469.95           0      456.79      426.04]
 [     83.024      445.49      57.035      13.153      456.79           0      65.391]
 [     117.78      396.83       28.16      73.246      426.04      65.391           0]]


In [313]:
unique_list = []
for group in groups_of_persons.keys():  # Исправлено на range(len(groups_of_persons)) для итерации по всем группам
    persons_in_group = np.array(groups_of_persons[group])
    unique_persons = np.unique(persons_in_group)
    unique_list.append(unique_persons)

# Вывод уникальных значений для каждой группы
print(unique_list[0])


[0 2 3 5 6]


In [328]:
list_of_coordinates=[]
for group in range(len(unique_list)): 
    list_of_x=[]
    list_of_y=[]
    for num in unique_list[group]:
        list_of_x.extend([objects["person"][num][0][0], objects["person"][0][0][2]])
        list_of_y.extend([objects["person"][num][0][1], objects["person"][0][0][3]])
    list_of_coordinates.append([list_of_x,list_of_y])
        
boundnoxes = []
for coordinates in list_of_coordinates:
    xmin, xmax = min(coordinates[0]), max(coordinates[0])
    ymin, ymax = min(coordinates[1]), max(coordinates[1])
    boundnoxes.append([xmin,ymin,xmax,ymax])
boundnoxes

[[146, 154, 255, 297]]

In [300]:
# Функция для обработки изображения
def process_image(image_path):
    # Загрузка изображения
    image = cv2.imread(image_path)
    results = model(image)[0]
    
    # Получение оригинального изображения и результатов
    image = results.orig_img
    classes_names = results.names
    classes = results.boxes.cls.cpu().numpy()
    boxes = results.boxes.xyxy.cpu().numpy().astype(np.int32)

    # Подготовка словаря для группировки результатов по классам
    grouped_objects = {}

    # Рисование рамок и группировка результатов
    for class_id, box in zip(classes, boxes):
        if class_id==0:    
            class_name = classes_names[int(class_id)]
            color = colors[int(class_id) % len(colors)]  # Выбор цвета для класса
            if class_name not in grouped_objects:
                grouped_objects[class_name] = []
                
            x1, y1, x2, y2 = box
            center_x = (x1 + x2) // 2
            center_y = (y1 + y2) // 2
            grouped_objects[class_name].append((box, (center_x, center_y)))
            
            cv2.rectangle(image, (x1, y1), (x2, y2), color, 2)
            cv2.putText(image, class_name, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

    # Сохранение измененного изображения
    new_image_path = os.path.splitext(image_path)[0] + '_yolo' + os.path.splitext(image_path)[1]
    cv2.imwrite(new_image_path, image)

    # Сохранение данных в текстовый файл
    text_file_path = os.path.splitext(image_path)[0] + '_data.txt'
    with open(text_file_path, 'w') as f:
        for class_name, details in grouped_objects.items():
            f.write(f"{class_name}:\n")
            for box, center in details:
                f.write(f"Coordinates: ({box[0]}, {box[1]}, {box[2]}, {box[3]}) Center: ({center[0]}, {center[1]})\n")

    count_of_persons = len([el for el in classes if el == 0])
    
    print(f"Count of persons: {count_of_persons}")
    print(f"Processed {image_path}:")
    print(f"Saved bounding-box image to {new_image_path}")
    print(f"Saved data to {text_file_path}")
    return results, grouped_objects

255